In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv(r'C:\Users\lenovo\vscode\Hangman_Project\Exploratory-Data-Analysis---Customer-Loans-in-Finance\loan_payments.csv')

In [ ]:
data

In [ ]:
data.head(5)

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
pd.set_option('display.max_rows', None)
data.isnull().sum()

In [ ]:
pd.set_option('display.max_rows', None)
data.info()

In [ ]:
null_percentage = (data.isnull().sum()/len(data)*100)
null_percentage 

In [ ]:
data.describe()

In [ ]:
data['term'].value_counts()

In [ ]:
data['grade'].value_counts()

In [ ]:
data['sub_grade'].value_counts()

In [ ]:
data['application_type'].value_counts()

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data.head(5)

In [ ]:
data['home_ownership'].value_counts()

In [ ]:
data['verification_status'].value_counts()

In [ ]:
data['loan_status'].value_counts()

In [ ]:
data['purpose'].value_counts()

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
null_percentage = (data.isnull().sum()/len(data)*100)
null_percentage 

In [ ]:
# Filter rows with null values in 'funded_amount'
null_rows_data = data[data['funded_amount'].isnull()]

In [ ]:
# Calculate the mean of 'funded_amount' (ignoring NaN values)
mean_value_f = data['funded_amount'].mean()

# Fill NaN values in 'funded_amount' with the mean
data['funded_amount'] = data['funded_amount'].fillna(mean_value_f)

In [ ]:
mean_value_f

In [ ]:
data.isnull().sum()

In [ ]:
data.head(5)

In [ ]:
# Calculate the mode of 'funded_amount' (the most frequent value)
mode_value = data['term'].mode()[0]

# Fill NaN values in 'funded_amount' with the mode
data['term'] = data['term'].fillna(mode_value)

In [ ]:
data.head(5)

In [ ]:
data.isnull().sum()

In [ ]:
import seaborn as sns
sns.heatmap(data.isnull())

In [ ]:
# Drop columns 'new' and 'month' from the DataFrame
data = data.drop(columns=['mths_since_last_delinq', 'mths_since_last_record', 'mths_since_last_major_derog'])

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
# Group by 'term' and calculate the mean of 'int_rate'
mean_int_rate = data.groupby('term')['int_rate'].mean()

In [ ]:
mean_int_rate

In [ ]:
# Means for each term
mean_36_months = 12.452135
mean_60_months = 16.635561

# Fill NaN in 'int_rate' based on 'term'
data.loc[(data['term'] == '36 months') & (data['int_rate'].isna()), 'int_rate'] = mean_36_months
data.loc[(data['term'] == '60 months') & (data['int_rate'].isna()), 'int_rate'] = mean_60_months

In [ ]:
data.isnull().sum()

In [ ]:
#fill the null data in next_payment_date with input of one year after last_payment_date
# Convert 'last_payment_date' to datetime objects
data['last_payment_date'] = pd.to_datetime(data['last_payment_date'])

# Now you can add the DateOffset
data['next_payment_date'] = data['last_payment_date'] + pd.DateOffset(years=1)
data.head()

In [ ]:
# Convert to datetime
data['next_payment_date'] = pd.to_datetime(data['next_payment_date'])

# Format to 'Jan-23'
data['next_payment_date'] = data['next_payment_date'].dt.strftime('%b-%y')

In [ ]:
# Convert to datetime
data['last_payment_date'] = pd.to_datetime(data['last_payment_date'])

# Format to 'Jan-23'
data['last_payment_date'] = data['last_payment_date'].dt.strftime('%b-%y')

In [ ]:
data.head(5)

In [ ]:
data.isnull().sum()

In [ ]:
null_percentage = (data.isnull().sum()/len(data)*100)
null_percentage 

In [ ]:
# Calculate the mode of 'funded_amount' (the most frequent value)
mode_value_e = data['employment_length'].mode()[0]

# Fill NaN values in 'funded_amount' with the mode
data['employment_length'] = data['employment_length'].fillna(mode_value_e)

In [ ]:
null_percentage = (data.isnull().sum()/len(data)*100)
null_percentage 

In [ ]:
data = data.dropna()

In [ ]:
data.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Calculate skewness
skewness = data.skew()

In [ ]:
# Define skewness threshold
threshold = 0.80

# Identify skewed columns
skewed_columns = skewness[skewness.abs() > threshold].index

In [ ]:

print("Skewed Columns:")
print(skewed_columns)

In [ ]:
for column in skewed_columns:
    sns.histplot(data[column], kde=True)
    plt.title(f"Distribution of {column} (Skew: {skewness[column]:.2f})")
    plt.show()

In [ ]:
import numpy as np
from scipy.stats import boxcox

class DataFrameTransform:
    def __init__(self, data):
        self.data = data

    def reduce_skew(self, column, method="log"):
        """
        Reduces skewness of a specified column using transformations.

        Parameters:
        - column (str): Column to transform.
        - method (str): Transformation method ('log', 'sqrt', 'boxcox', 'log1p').

        Returns:
        - Transformed column (pd.Series).
        """
        col_data = self.data[column]

        if method == "log":
            transformed = np.log(col_data[col_data > 0])
        elif method == "sqrt":
            transformed = np.sqrt(col_data[col_data >= 0])
        elif method == "boxcox":
            if col_data.min() <= 0:
                raise ValueError("Box-Cox requires strictly positive values.")
            transformed, _ = boxcox(col_data)
        elif method == "log1p":
            transformed = np.log1p(col_data[col_data >= 0])
        else:
            raise ValueError("Invalid method. Choose from 'log', 'sqrt', 'boxcox', 'log1p'.")

        return transformed

    def auto_transform_skewed(self, threshold=0.75):
        """
        Identifies skewed columns and applies transformations to reduce skewness.

        Parameters:
        - threshold (float): Skewness threshold above which a column is considered skewed.

        Returns:
        - Updated DataFrame with transformed columns.
        """
        skewness = self.data.skew()
        skewed_columns = skewness[skewness.abs() > threshold].index

        print(f"Skewed Columns: {skewed_columns}")
        
        for column in skewed_columns:
            try:
                print(f"Transforming column: {column}")
                self.data[column] = self.reduce_skew(column, method="log1p")
            except Exception as e:
                print(f"Error transforming column {column}: {e}")

        return self.data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import probplot
def visualize_skewness(self, column, before_data, after_data):
        """
        Visualize the skewness of a column before and after transformation.

        Parameters:
        - column (str): Column name.
        - before_data (pd.Series): Column data before transformation.
        - after_data (pd.Series): Column data after transformation.
        """
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))

        # Histogram comparison
        sns.histplot(before_data, kde=True, ax=axes[0], color='blue', label='Before')
        sns.histplot(after_data, kde=True, ax=axes[0], color='orange', label='After')
        axes[0].set_title(f"Histogram: {column}")
        axes[0].legend()

        # Boxplot comparison
        sns.boxplot(data=[before_data.dropna(), after_data.dropna()], ax=axes[1])
        axes[1].set_xticklabels(['Before', 'After'])
        axes[1].set_title(f"Boxplot: {column}")

        # Q-Q plot comparison
        probplot(before_data.dropna(), dist="norm", plot=axes[2])
        probplot(after_data.dropna(), dist="norm", plot=axes[2])
        axes[2].set_title(f"Q-Q Plot: {column}")

        plt.tight_layout()
        plt.show()

In [ ]:
def visualize_skewness(self, column, before_data, after_data):
    """
    Visualize the skewness of a column before and after transformation.
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Histogram comparison
    sns.histplot(before_data, kde=True, ax=axes[0], color='blue', label='Before')
    sns.histplot(after_data, kde=True, ax=axes[0], color='orange', label='After')
    axes[0].set_title(f"Histogram: {column}")
    axes[0].legend()

    # Boxplot comparison
    sns.boxplot(data=[before_data.dropna(), after_data.dropna()], ax=axes[1])
    axes[1].set_xticklabels(['Before', 'After'])
    axes[1].set_title(f"Boxplot: {column}")

    # Q-Q plot comparison
    probplot(before_data.dropna(), dist="norm", plot=axes[2])
    probplot(after_data.dropna(), dist="norm", plot=axes[2])
    axes[2].set_title(f"Q-Q Plot: {column}")

    plt.tight_layout()
    plt.show()  # Ensure plots are displayed

In [ ]:
sns.boxplot(x=data['total_rec_int'])

In [ ]:
# Plot the box plot
plt.figure(figsize=(8, 6))
plt.xticks(rotation=90)
sns.boxplot(data=data, x='term', y='int_rate', color="skyblue", boxprops=dict(facecolor="pink"))

In [ ]:
# Plot the box plot
plt.figure(figsize=(8, 6))
plt.xticks(rotation=90)
sns.boxplot(data=data, x='term', y='loan_amount', color="skyblue", boxprops=dict(facecolor="pink"))

In [ ]:
sns.boxplot(x=data['loan_amount'])

In [ ]:
sns.boxplot(x=data['total_payment'])

In [ ]:
sns.boxplot(x=data['out_prncp'])

In [ ]:
# Plot the box plot
plt.figure(figsize=(8, 6))
plt.xticks(rotation=90)
sns.boxplot(data=data, x='home_ownership', y='annual_inc', color="skyblue", boxprops=dict(facecolor="pink"))

In [ ]:
sns.boxplot(x=data['annual_inc'])

In [ ]:
import numpy as np

class DataFrameTransform:
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def handle_outliers(self, columns, method="remove", threshold=1.5):
        """
        Handles outliers in the specified columns based on the chosen method.

        Parameters:
        - columns (list): List of column names to process.
        - method (str): Method to handle outliers:
            - "remove": Remove rows with outliers.
            - "replace_with_median": Replace outliers with the median.
            - "replace_with_mean": Replace outliers with the mean.
        - threshold (float): Threshold for detecting outliers (e.g., IQR multiplier).
        
        Returns:
        - pd.DataFrame: DataFrame with outliers handled.
        """
        for column in columns:
            if column not in self.dataframe:
                print(f"Column '{column}' not found in DataFrame.")
                continue

            # Compute IQR
            Q1 = self.dataframe[column].quantile(0.25)
            Q3 = self.dataframe[column].quantile(0.75)
            IQR = Q3 - Q1

            # Define outlier bounds
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR

            # Identify outliers
            is_outlier = (self.dataframe[column] < lower_bound) | (self.dataframe[column] > upper_bound)

            if method == "remove":
                # Remove rows with outliers
                self.dataframe = self.dataframe[~is_outlier]
            elif method == "replace_with_median":
                # Replace outliers with the median
                median_value = self.dataframe[column].median()
                self.dataframe.loc[is_outlier, column] = median_value
            elif method == "replace_with_mean":
                # Replace outliers with the mean
                mean_value = self.dataframe[column].mean()
                self.dataframe.loc[is_outlier, column] = mean_value
            else:
                raise ValueError(f"Invalid method: {method}. Choose 'remove', 'replace_with_median', or 'replace_with_mean'.")

        return self.dataframe


In [ ]:
transformer = DataFrameTransform(data)

# Handle outliers in 'feature1' and 'feature2' by removing them
data_cleaned = transformer.handle_outliers(columns=['out_prncp', 'total_rec_int', 'last_payment_amount', 'annual_inc', 'loan_amount', 'funded_amount'], method="remove")
print("\nData After Removing Outliers:")
print(data_cleaned)

# Handle outliers by replacing with the median
data_median = transformer.handle_outliers(columns=['out_prncp', 'total_rec_int', 'last_payment_amount', 'annual_inc', 'loan_amount', 'funded_amount'], method="replace_with_median")
print("\nData After Replacing Outliers with Median:")
print(data_median)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compare original vs cleaned data
sns.histplot(data['loan_amount'], kde=True, color="blue", label="Original")
sns.histplot(data_cleaned['loan_amount'], kde=True, color="orange", label="Cleaned")
plt.legend()
plt.title("Outlier Handling in Feature1")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compare original vs cleaned data
sns.histplot(data['last_payment_amount'], kde=True, color="blue", label="Original")
sns.histplot(data_cleaned['last_payment_amount'], kde=True, color="orange", label="Cleaned")
plt.legend()
plt.title("Outlier Handling in last_payment_amount")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compare original vs cleaned data
sns.histplot(data['funded_amount'], kde=True, color="blue", label="Original")
sns.histplot(data_cleaned['funded_amount'], kde=True, color="orange", label="Cleaned")
plt.legend()
plt.title("Outlier Handling in last_payment_amount")
plt.show()

In [ ]:
# Compute the correlation matrix
correlation_matrix = data.corr()

# Visualize the correlation matrix
plt.figure(figsize=(20, 20))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

In [ ]:
def find_highly_correlated_columns(correlation_matrix, threshold=0.85):
    """
    Finds pairs of columns with correlations above a given threshold.

    Parameters:
    - correlation_matrix (pd.DataFrame): Correlation matrix of the dataset.
    - threshold (float): Correlation threshold.

    Returns:
    - list: List of tuples representing highly correlated column pairs.
    """
    correlated_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i + 1, len(correlation_matrix.columns)):
            if abs(correlation_matrix.iloc[i, j]) > threshold:
                col1 = correlation_matrix.columns[i]
                col2 = correlation_matrix.columns[j]
                correlated_pairs.append((col1, col2))
    return correlated_pairs

# Identify highly correlated column pairs
highly_correlated = find_highly_correlated_columns(correlation_matrix, threshold=0.85)
print("Highly Correlated Pairs:")
print(highly_correlated)

In [ ]:
def select_columns_to_remove(correlated_pairs):
    """
    Selects columns to remove from a list of highly correlated column pairs.

    Parameters:
    - correlated_pairs (list): List of tuples with correlated column pairs.

    Returns:
    - set: Set of columns to remove.
    """
    columns_to_remove = set()
    for col1, col2 in correlated_pairs:
        # Keep one column and mark the other for removal (arbitrary choice here)
        columns_to_remove.add(col2)
    return columns_to_remove

# Get columns to remove
columns_to_remove = select_columns_to_remove(highly_correlated)
print("Columns to Remove:")
print(columns_to_remove)


In [ ]:
# Drop the identified columns
data_reduced = data.drop(columns=columns_to_remove)
print("Dataset after removing highly correlated columns:")
print(data_reduced)
